In [103]:
from langchain_classic.retrievers import BM25Retriever, EnsembleRetriever

class PolicyRetrievers:
    """
    청년정책 검색기를 생성하는 클래스.
    - BM25Retriever
    - Vector Retriever
    - EnsembleRetriever (BM25 + Vector)
    - 사용자 정보 필터링 (나이, 지역, 학력)
    """

    def __init__(self, documents, vectorstore, 
                 bm25_k=5, vector_k=10, 
                 bm25_weight=0.6, vector_weight=0.4):

        self.documents = documents
        self.vectorstore = vectorstore

        # 파라미터 저장
        self.bm25_k = bm25_k
        self.vector_k = vector_k
        self.bm25_weight = bm25_weight
        self.vector_weight = vector_weight

        # 사용자 정보 초기화
        self.user_age = None
        self.user_region = None
        self.user_education = None

        # 초기화
        self._build_bm25()
        self._build_vector()
        self._build_ensemble()

    # --------------------------
    # 사용자 정보 설정
    # --------------------------
    def set_user_info(self, age=None, region=None, education=None):
        """
        사용자 정보 설정
        
        Args:
            age: 나이 (예: 25)
            region: 지역 (예: "서울특별시 강남구", "경기도 의정부시")
            education: 학력 (예: "대학교 재학", "고등학교 졸업")
        """
        self.user_age = age
        self.user_region = region
        self.user_education = education
        
        info = []
        if age:
            info.append(f"나이 {age}세")
        if region:
            info.append(f"지역 {region}")
        if education:
            info.append(f"학력 {education}")
        
        if info:
            print(f"✅ 사용자 정보 설정: {', '.join(info)}")

    # --------------------------
    # BM25 Retriever
    # --------------------------
    def _build_bm25(self):
        self.bm25_retriever = BM25Retriever.from_documents(self.documents)
        self.bm25_retriever.k = self.bm25_k

    # --------------------------
    # Vector Retriever (FAISS/Chroma 등)
    # --------------------------
    def _build_vector(self):
        self.vector_retriever = self.vectorstore.as_retriever(
            search_type="similarity",
            search_kwargs={"k": self.vector_k},
        )

    # --------------------------
    # Ensemble Retriever (BM25 + Vector)
    # --------------------------
    def _build_ensemble(self):
        self.ensemble_retriever = EnsembleRetriever(
            retrievers=[self.bm25_retriever, self.vector_retriever],
            weights=[self.bm25_weight, self.vector_weight],
        )

    # --------------------------
    # 사용자 정보 기반 필터링
    # --------------------------
    def filter_by_user_info(self, documents):
        """
        검색 결과를 사용자 정보(나이, 지역, 학력)로 필터링
        
        Args:
            documents: 검색된 문서 리스트
            
        Returns:
            list: 필터링된 문서 리스트
        """
        # 사용자 정보가 없으면 필터링 없이 반환
        if not (self.user_age or self.user_region or self.user_education):
            return documents
        
        filtered = []
        
        for doc in documents:
            metadata = doc.metadata
            
            # 1. 나이 필터링
            age_match = True
            if self.user_age:
                try:
                    min_age = int(metadata.get('지원최소연령', '0') or '0')
                    max_age = int(metadata.get('지원최대연령', '0') or '0')
                    
                    if min_age > 0 and self.user_age < min_age:
                        age_match = False
                    if max_age > 0 and max_age < 999 and self.user_age > max_age:
                        age_match = False
                except:
                    pass
            
            # 2. 지역 필터링 (쉼표 구분 단위로 정확히 매칭)
            region_match = True
            if self.user_region:
                policy_region = metadata.get('지역', '')  # 실제 정책 적용 지역
                
                # 지역 필드가 있으면 반드시 확인
                if policy_region:
                    region_match = False
                    
                    # 정책 지역을 쉼표로 분리 (각 지역이 개별 항목)
                    policy_regions = [r.strip() for r in policy_region.split(',')]
                    
                    # 시/도 추출
                    sido_list = ['서울', '경기', '인천', '부산', '대구', '광주', 
                                 '대전', '울산', '세종', '강원', '충북', '충남', 
                                 '전북', '전남', '경북', '경남', '제주']
                    
                    user_sido = None
                    for sido in sido_list:
                        if sido in self.user_region:
                            user_sido = sido
                            break
                    
                    # 구/군/시 추출
                    region_parts = self.user_region.split()
                    user_district = region_parts[-1] if len(region_parts) > 1 else None
                    
                    # 각 정책 지역과 비교
                    for pr in policy_regions:
                        # 1) 정확히 일치하는 경우 (예: "경상남도 남해군" == "경상남도 남해군")
                        if self.user_region == pr:
                            region_match = True
                            break
                        
                        # 2) 사용자 지역이 정책 지역에 포함되는 경우
                        # 예: 사용자 "경상남도 남해군", 정책 "경상남도 남해군"
                        if self.user_region in pr:
                            region_match = True
                            break
                        
                        # 3) 시/도 + 구/군 매칭
                        if user_sido and user_district:
                            # 정책 지역에 시/도와 구/군이 모두 있는지 확인
                            if user_sido in pr and user_district in pr:
                                region_match = True
                                break
                else:
                    # 지역 필드가 없으면 제외
                    region_match = False
            
            # 3. 학력 필터링
            education_match = True
            if self.user_education:
                edu_req = metadata.get('학력요건', '')
                
                if '제한없음' in edu_req or '기타' in edu_req:
                    education_match = True
                else:
                    education_match = False
                    
                    if "고등학교" in self.user_education:
                        if any(kw in edu_req for kw in ["고등", "고졸", "고교"]):
                            education_match = True
                    
                    elif "대학교" in self.user_education:
                        if any(kw in edu_req for kw in ["대학", "학사", "재학"]):
                            education_match = True
                    
                    elif "대학원" in self.user_education:
                        education_match = True  # 최고 학력
            
            # 세 조건 모두 만족하면 포함
            if age_match and region_match and education_match:
                filtered.append(doc)
        
        print(f"📊 필터링: {len(documents)}개 → {len(filtered)}개")
        return filtered

    # --------------------------
    # 쿼리 증강 (사용자 정보를 쿼리에 추가)
    # --------------------------
    def _enhance_query(self, query):
        """
        사용자 정보를 활용해 검색 쿼리 증강
        
        Args:
            query: 원본 검색 쿼리
            
        Returns:
            str: 증강된 검색 쿼리
        """
        enhanced = query
        
        # 지역 정보 추가
        if self.user_region:
            # "서울특별시 강동구" → "강동구" 추출
            region_parts = self.user_region.split()
            district = region_parts[-1] if len(region_parts) > 1 else self.user_region
            
            # 쿼리에 지역이 없으면 추가
            if district not in query:
                enhanced = f"{query} {district}"
        
        if enhanced != query:
            print(f"🔍 쿼리 증강: '{query}' → '{enhanced}'")
        
        return enhanced

    # --------------------------
    # 외부에서 가져오기
    # --------------------------
    def get_bm25(self):
        return self.bm25_retriever

    def get_vector(self):
        return self.vector_retriever

    def get_ensemble(self):
        return self.ensemble_retriever
    
    def get_filtered_results(self, query, retriever_type="ensemble", enhance_query=True):
        """
        검색 + 사용자 정보 필터링을 한 번에 수행
        
        Args:
            query: 검색 쿼리
            retriever_type: "bm25", "vector", "ensemble" 중 선택
            enhance_query: 사용자 정보로 쿼리 증강 여부 (기본값: True)
            
        Returns:
            list: 필터링된 검색 결과
        """
        # 쿼리 증강 (사용자 지역 정보 자동 추가)
        if enhance_query:
            search_query = self._enhance_query(query)
        else:
            search_query = query
        
        # 검색 수행
        if retriever_type == "bm25":
            results = self.bm25_retriever.invoke(search_query)
        elif retriever_type == "vector":
            results = self.vector_retriever.invoke(search_query)
        else:  # ensemble
            results = self.ensemble_retriever.invoke(search_query)
        
        # 사용자 정보로 필터링
        filtered = self.filter_by_user_info(results)
        
        return filtered

# 사용 예시

PolicyRetrievers 클래스를 사용하여 청년 정책을 검색하고 필터링하는 예시입니다.

In [104]:
# 1. 필요한 라이브러리 임포트
import os
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
import chromadb

# 환경 변수 로드
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

print("✅ 라이브러리 임포트 완료")

✅ 라이브러리 임포트 완료


In [105]:
# 2. ChromaDB에서 문서 로드
db_path = "../data/vectordb"
current_dir = os.path.dirname(os.path.abspath("__file__"))
project_root = os.path.dirname(current_dir)
full_db_path = os.path.join(project_root, "data", "vectordb")

print(f"📁 벡터DB 경로: {full_db_path}")

# 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    api_key=OPENAI_API_KEY
)

# ChromaDB 연결
vectorstore = Chroma(
    persist_directory=full_db_path,
    collection_name="youth_policies",
    embedding_function=embeddings
)

# ChromaDB collection 접근
chroma_client = chromadb.PersistentClient(path=full_db_path)
collection = chroma_client.get_collection(name="youth_policies")

# 문서 로드
all_data = collection.get()
documents = []
for doc_text, metadata in zip(all_data['documents'], all_data['metadatas']):
    documents.append(Document(
        page_content=doc_text,
        metadata=metadata
    ))

print(f"✅ 문서 로드 완료: {len(documents)}개")

📁 벡터DB 경로: c:\Users\31799\Desktop\SKN20-3rd-1TEAM\data\vectordb
✅ 문서 로드 완료: 3550개
✅ 문서 로드 완료: 3550개


In [106]:
# 3. PolicyRetrievers 초기화
retrievers = PolicyRetrievers(
    documents=documents,
    vectorstore=vectorstore,
    bm25_k=10,           # BM25 검색 결과 수
    vector_k=10,         # Vector 검색 결과 수
    bm25_weight=0.2,     # BM25 가중치
    vector_weight=0.8    # Vector 가중치
)

print("✅ PolicyRetrievers 초기화 완료")

✅ PolicyRetrievers 초기화 완료


In [118]:
# 4. 사용자 정보 설정
retrievers.set_user_info(
    age=25,
    region="서울특별시 강동구",
    education=None
)

✅ 사용자 정보 설정: 나이 25세, 지역 서울특별시 강동구


In [ ]:
# 5. 검색 + 필터링 (Ensemble 방식)
query = "창업 컨설팅 서울특별시"

print(f"\n🔍 원본 검색어: {query}")
print("="*70)

# 쿼리 증강 + 검색 + 필터링 (한 번에)
results = retrievers.get_filtered_results(query, retriever_type="ensemble")
print(f"\n📊 최종 결과: {len(results)}개\n")

# 결과 출력
for i, doc in enumerate(results, 1):
    metadata = doc.metadata
    policy_name = metadata.get('정책명', 'N/A')
    
    # 타겟 정책 강조
    if '2024 창업취약분야 무료 컨설팅' in policy_name:
        print(f"✅ [{i}] {policy_name} ⭐⭐⭐ 타겟 정책!")
    else:
        print(f"[{i}] {policy_name}")
    
    print(f"    분야: {metadata.get('중분류', 'N/A')}")
    print(f"    담당: {metadata.get('주관기관명', 'N/A')}")
    print(f"    연령: {metadata.get('지원최소연령', 'N/A')}세 ~ {metadata.get('지원최대연령', 'N/A')}세")
    print(f"    지역: {metadata.get('지역', '전국')}")
    print()
    
    if i >= 10:  # 상위 10개만 출력
        break


🔍 원본 검색어: 창업 컨설팅 서울특별시
🔍 쿼리 증강: '창업 컨설팅 서울특별시' → '창업 컨설팅 서울특별시 강동구'
📊 필터링: 17개 → 16개

📊 최종 결과: 16개

[1] 청년해냄센터 운영
    분야: 창업
    담당: 강동구청 기획경제국 일자리정책과
    연령: 19세 ~ 39세
    지역: 서울특별시 강동구

[2] 엔젤공방거리 조성
    분야: 창업
    담당: 강동구청 기획경제국 일자리정책과
    연령: 19세 ~ 39세
    지역: 서울특별시 강동구

[3] 구천면로 청년다락공방 운영
    분야: 창업
    담당: 강동구청 기획경제국 일자리정책과
    연령: 19세 ~ 39세
    지역: 서울특별시 강동구

[4] 창업보육센터 지원
    분야: 창업
    담당: 서산시
    연령: 18세 ~ 39세
    지역: 서울특별시 종로구, 서울특별시 중구, 서울특별시 용산구, 서울특별시 성동구, 서울특별시 광진구, 서울특별시 동대문구, 서울특별시 중랑구, 서울특별시 성북구, 서울특별시 강북구, 서울특별시 도봉구, 서울특별시 노원구, 서울특별시 은평구, 서울특별시 서대문구, 서울특별시 마포구, 서울특별시 양천구, 서울특별시 강서구, 서울특별시 구로구, 서울특별시 금천구, 서울특별시 영등포구, 서울특별시 동작구, 서울특별시 관악구, 서울특별시 서초구, 서울특별시 강남구, 서울특별시 송파구, 서울특별시 강동구, 부산광역시 중구, 부산광역시 서구, 부산광역시 동구, 부산광역시 영도구, 부산광역시 부산진구, 부산광역시 동래구, 부산광역시 남구, 부산광역시 북구, 부산광역시 해운대구, 부산광역시 사하구, 부산광역시 금정구, 부산광역시 강서구, 부산광역시 연제구, 부산광역시 수영구, 부산광역시 사상구, 부산광역시 기장군, 대구광역시 중구, 대구광역시 동구, 대구광역시 서구, 대구광역시 남구, 대구광역시 북구, 대구광역시 수성구, 대구광역시 달서구, 대구광역시 달성군, 대구광역시 군위군, 인천광역시 중구, 인천광역시 동구, 인천광

## 다른 검색 방식 비교

BM25, Vector, Ensemble 세 가지 방식을 비교해봅니다.

In [109]:
# 6. 검색 방식 비교 (BM25 vs Vector vs Ensemble)
query = "청년 창업 자금 지원"

print(f"🔍 검색어: {query}\n")

for retriever_type in ["bm25", "vector", "ensemble"]:
    results = retrievers.get_filtered_results(query, retriever_type=retriever_type)
    print(f"\n{'='*50}")
    print(f"📊 {retriever_type.upper()} 검색 결과: {len(results)}개")
    print(f"{'='*50}")
    
    for i, doc in enumerate(results[:3], 1):
        metadata = doc.metadata
        print(f"[{i}] {metadata.get('정책명', 'N/A')}")
        print(f"    분야: {metadata.get('중분류', 'N/A')}")
        print()

🔍 검색어: 청년 창업 자금 지원

🔍 쿼리 증강: '청년 창업 자금 지원' → '청년 창업 자금 지원 강동구'
📊 필터링: 10개 → 4개

📊 BM25 검색 결과: 4개
[1] 로컬크리에이터 양성(로컬인서울)
    분야: 창업

[2] 엔젤공방거리 조성
    분야: 창업

[3] 청년해냄센터 운영
    분야: 창업

🔍 쿼리 증강: '청년 창업 자금 지원' → '청년 창업 자금 지원 강동구'
📊 필터링: 10개 → 5개

📊 VECTOR 검색 결과: 5개
[1] 청년해냄센터 운영
    분야: 창업

[2] 구천면로 청년다락공방 운영
    분야: 창업

[3] 청년전용창업자금
    분야: 창업

🔍 쿼리 증강: '청년 창업 자금 지원' → '청년 창업 자금 지원 강동구'
📊 필터링: 10개 → 5개

📊 VECTOR 검색 결과: 5개
[1] 청년해냄센터 운영
    분야: 창업

[2] 구천면로 청년다락공방 운영
    분야: 창업

[3] 청년전용창업자금
    분야: 창업

🔍 쿼리 증강: '청년 창업 자금 지원' → '청년 창업 자금 지원 강동구'
📊 필터링: 16개 → 7개

📊 ENSEMBLE 검색 결과: 7개
[1] 청년해냄센터 운영
    분야: 창업

[2] 강동구 미취업 청년 어학 및 자격시험 응시료 지원
    분야: 취업

[3] 구천면로 청년다락공방 운영
    분야: 창업

📊 필터링: 16개 → 7개

📊 ENSEMBLE 검색 결과: 7개
[1] 청년해냄센터 운영
    분야: 창업

[2] 강동구 미취업 청년 어학 및 자격시험 응시료 지원
    분야: 취업

[3] 구천면로 청년다락공방 운영
    분야: 창업



In [110]:
# 디버깅: 필터링 전/후 비교
query = "창업 컨설팅 강동구"

print(f"🔍 검색어: {query}\n")
print("="*70)
print("1️⃣ 필터링 전 (원본 검색 결과)")
print("="*70)

# 필터링 없이 검색
raw_results = retrievers.ensemble_retriever.invoke(query)
print(f"검색 결과: {len(raw_results)}개\n")

# 타겟 정책 찾기
target_found = False
for i, doc in enumerate(raw_results, 1):
    policy_name = doc.metadata.get('정책명', 'N/A')
    if '2024 창업취약분야 무료 컨설팅' in policy_name:
        print(f"✅ [{i}] {policy_name} ⭐ 타겟 정책 발견!")
        print(f"    지역: {doc.metadata.get('지역', 'N/A')}")
        print(f"    나이: {doc.metadata.get('지원최소연령', 'N/A')} ~ {doc.metadata.get('지원최대연령', 'N/A')}")
        target_found = True
        break

if not target_found:
    print("❌ 타겟 정책이 원본 검색 결과에 없습니다.")
    print("   → 검색 k 값을 늘리거나 검색어를 변경해보세요.\n")

print("\n" + "="*70)
print("2️⃣ 필터링 후 (나이 25세, 서울특별시 강동구)")
print("="*70)

# 필터링 적용
filtered_results = retrievers.get_filtered_results(query, retriever_type="ensemble")

# 타겟 정책 찾기
for i, doc in enumerate(filtered_results, 1):
    policy_name = doc.metadata.get('정책명', 'N/A')
    if '2024 창업취약분야 무료 컨설팅' in policy_name:
        print(f"✅ [{i}] {policy_name} ⭐ 타겟 정책!")
        print(f"    지역: {doc.metadata.get('지역', 'N/A')}")
        break

🔍 검색어: 창업 컨설팅 강동구

1️⃣ 필터링 전 (원본 검색 결과)
검색 결과: 16개

✅ [2] 2024 창업취약분야 무료 컨설팅(강동구청년해냄센터) ⭐ 타겟 정책 발견!
    지역: 서울특별시 강동구
    나이: 0 ~ 0

2️⃣ 필터링 후 (나이 25세, 서울특별시 강동구)
검색 결과: 16개

✅ [2] 2024 창업취약분야 무료 컨설팅(강동구청년해냄센터) ⭐ 타겟 정책 발견!
    지역: 서울특별시 강동구
    나이: 0 ~ 0

2️⃣ 필터링 후 (나이 25세, 서울특별시 강동구)
📊 필터링: 16개 → 6개
✅ [2] 2024 창업취약분야 무료 컨설팅(강동구청년해냄센터) ⭐ 타겟 정책!
    지역: 서울특별시 강동구
📊 필터링: 16개 → 6개
✅ [2] 2024 창업취약분야 무료 컨설팅(강동구청년해냄센터) ⭐ 타겟 정책!
    지역: 서울특별시 강동구
